# Scrapping weather related data

In [ ]:
!pip install selenium

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.7/9.7 MB 68.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 481.7/481.7 kB 25.5 MB/s eta 0:00:00


In [ ]:
import pandas as pd
import numpy as np
from selenium import webdriver
import time

In [ ]:
races = pd.read_csv('races.csv')
races.head()

,season,round,circuit_id,lat,long,country,date,url
0,1980,1,galvez,-34.6943,-58.45930,Argentina,1980-01-13,http://en.wikipedia.org/wiki/1980_Argentine_Gr...
1,1980,2,interlagos,-23.7036,-46.69970,Brazil,1980-01-27,http://en.wikipedia.org/wiki/1980_Brazilian_Gr...
2,1980,3,kyalami,-25.9894,28.07670,South Africa,1980-03-01,http://en.wikipedia.org/wiki/1980_South_Africa...
3,1980,4,long_beach,33.7651,-118.18900,USA,1980-03-30,http://en.wikipedia.org/wiki/1980_United_State...
4,1980,5,zolder,50.9894,5.25694,Belgium,1980-05-04,http://en.wikipedia.org/wiki/1980_Belgian_Gran...


In [ ]:
races.shape

(751, 8)

In [ ]:
weather = races.iloc[:,[0,1,2]]

In [ ]:
info = []

for link in races.url:
    try:
        df = pd.read_html(link)[0]
        if 'Weather' in list(df.iloc[:,0]):
            n = list(df.iloc[:,0]).index('Weather')
            info.append(df.iloc[n,1])
        else:
            df = pd.read_html(link)[1]
            if 'Weather' in list(df.iloc[:,0]):
                n = list(df.iloc[:,0]).index('Weather')
                info.append(df.iloc[n,1])
            else:
                df = pd.read_html(link)[2]
                if 'Weather' in list(df.iloc[:,0]):
                    n = list(df.iloc[:,0]).index('Weather')
                    info.append(df.iloc[n,1])
                else:
                    df = pd.read_html(link)[3]
                    if 'Weather' in list(df.iloc[:,0]):
                        n = list(df.iloc[:,0]).index('Weather')
                        info.append(df.iloc[n,1])
                    else:
                        driver = webdriver.Chrome()
                        driver.get(link)

                        # click language button
                        button = driver.find_element_by_link_text('Italiano')
                        button.click()

                        clima = driver.find_element_by_xpath('//*[@id="mw-content-text"]/div/table[1]/tbody/tr[9]/td').text
                        info.append(clima)
    except:
        info.append('not found')

In [ ]:
len(info)

751

In [ ]:
weather['weather'] = info

<ipython-input-8-0345b68f31a0>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  weather['weather'] = info


In [ ]:
weather.head()

,season,round,circuit_id,weather
0,1980,1,galvez,"Sunny, Very hot, Dry"
1,1980,2,interlagos,Dry
2,1980,3,kyalami,Dry
3,1980,4,long_beach,Sunny and warm with temperatures reaching up t...
4,1980,5,zolder,Dry


In [ ]:
weather.tail()

,season,round,circuit_id,weather
746,2022,18,suzuka,Rain
747,2022,19,americas,Partly cloudy
748,2022,20,rodriguez,Partly cloudy
749,2022,21,interlagos,Cloudy
750,2022,22,yas_marina,Clear


In [ ]:
weather['weather'].unique()

array(['Sunny, Very hot, Dry', 'Dry',
       'Sunny and warm with temperatures reaching up to 81\xa0°F (27\xa0°C); wind speeds up to 10.9 miles per hour (17.5\xa0km/h)[1]',
       'Dry at the start, damp in the end', 'not found', 'Overcast, Dry',
       'Mild, Dry', 'Sunny, Mild, Dry',
       'Cold and dry with temperatures reaching up to 11\xa0°C (52\xa0°F); wind speeds up to 15.9 kilometres per hour (9.9\xa0mph)[1]',
       'Cloudy and dry with temperatures reaching up to 12\xa0°C (54\xa0°F); Wind speeds up to 14.82\xa0km/h (9.21\xa0mph)[1]',
       'Sunny and warm with temperatures reaching up to 71.1\xa0°F (21.7\xa0°C); wind speeds approaching 10.9 miles per hour (17.5\xa0km/h)[1]',
       'Overcast, Fresh, Rain', 'Sunny, dry', 'Rain, cold',
       'Dry, then wet at the end', 'Sunny, hot',
       'Dry, then wet at half distance, then dry again', 'Wet / Dry',
       'Dry and sunny',
       'Wet and cold with temperatures up to 20\xa0°C (68\xa0°F); wind speeds up to 15 kilometres per

In [ ]:
class WeatherClassifier:
    def __init__(self, weather_dict):
        """
        Initialize the WeatherClassifier with a dictionary of weather categories and keywords.

        Args:
            weather_dict (dict): A dictionary where keys are weather categories and values are lists of keywords.
        """
        self.weather_dict = weather_dict

    def narrow_down_weather(self, description):
        """
        Classify a weather description into one or more categories based on keywords in the weather dictionary.

        Args:
            description (str): A description of the weather.

        Returns:
            dict: A dictionary where keys are weather categories and values are the matched keywords.
        """
        description_lower = description.lower()
        matched_categories = {}

        for category, keywords in self.weather_dict.items():
            matches = [keyword for keyword in keywords if keyword.lower() in description_lower]
            if matches:
                matched_categories[category] = matches

        return matched_categories if matched_categories else {"weather_unknown": []}

In [ ]:
weather_dict = {
    'weather_warm': ['warm', 'hot', 'sunny', 'mild', 'bright', 'pleasant'],
    'weather_cold': ['cold', 'chilly', 'freezing', 'frosty', 'cool'],
    'weather_dry': ['dry', 'arid', 'parched'],
    'weather_wet': ['wet', 'rain', 'rainy', 'showers', 'drizzle', 'thunderstorms', 'stormy'],
    'weather_cloudy': ['cloudy', 'overcast', 'grey', 'gloomy', 'misty']
}


In [ ]:
weather_df = pd.DataFrame(columns = weather_dict.keys())

In [ ]:
for col in weather_df:
    weather_df[col] = weather['weather'].map(lambda x: 1 if any(i in weather_dict[col] for i in x.lower().split()) else 0)

In [ ]:
weather_df.head()

,weather_warm,weather_cold,weather_dry,weather_wet,weather_cloudy
0,0,0,1,0,0
1,0,0,1,0,0
2,0,0,1,0,0
3,1,0,0,0,0
4,0,0,1,0,0


In [ ]:
weather_info = pd.concat([weather, weather_df], axis = 1)

In [ ]:
weather_info.shape

(751, 9)

In [ ]:
weather_info

,season,round,circuit_id,weather,weather_warm,weather_cold,weather_dry,weather_wet,weather_cloudy
0,1980,1,galvez,"Sunny, Very hot, Dry",0,0,1,0,0
1,1980,2,interlagos,Dry,0,0,1,0,0
2,1980,3,kyalami,Dry,0,0,1,0,0
3,1980,4,long_beach,Sunny and warm with temperatures reaching up t...,1,0,0,0,0
4,1980,5,zolder,Dry,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...
746,2022,18,suzuka,Rain,0,0,0,1,0
747,2022,19,americas,Partly cloudy,0,0,0,0,1
748,2022,20,rodriguez,Partly cloudy,0,0,0,0,1
749,2022,21,interlagos,Cloudy,0,0,0,0,1


In [ ]:
weather_info.to_csv('weather.csv', index= False)